In [3]:
'''
Este programa permite analizar los movimientos de entrada/salida del SCI. El objetivo de este
programa es organizar los artículos según su porcentaje de salida, y así identificar los que tienen
más movimientos. 

Se obtiene un cuociente entre las Salidas/Entradas. Este es un porcentaje de movimientos. Por 
ejemplo:

- Un Valor de 0.86 significa que el 86% de los artículos han salido.

Para este efecto, se genera una tabla del estilo:
Codigo_Articulo | Nombre_Articulo | Entradas | Salidas

Lo anterior se logra agrupando las tablas según su código. Los códigos que SÓlO tienen salidas, y 
0 entradas, significa que en años anteriores tuvieron una entrada. Estos casos serán revisados 
después.
'''

'\nEste programa permite analizar los movimientos de entrada/salida del SCI. El objetivo de este\nprograma es organizar los artículos según su porcentaje de salida, y así identificar los que tienen\nmás movimientos. \n\nSe obtiene un cuociente entre las Salidas/Entradas. Este es un porcentaje de movimientos. Por \nejemplo:\n\n- Un Valor de 0.86 significa que el 86% de los artículos han salido.\n\nPara este efecto, se genera una tabla del estilo:\nCodigo_Articulo | Nombre_Articulo | Entradas | Salidas\n\nLo anterior se logra agrupando las tablas según su código. Los códigos que SÓlO tienen salidas, y \n0 entradas, significa que en años anteriores tuvieron una entrada. Estos casos serán revisados \ndespués.\n'

In [4]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
pd.options.display.float_format = '{:,.2f}'.format


In [5]:
def obtener_full_path(directorio):
    return [os.path.join(directorio, file) for file in os.listdir(directorio)]

def separar_por_cuartil(df_a_separar, columna_a_separar):
    qmin = df_a_separar[columna_a_separar].quantile(0)
    q1 = df_a_separar[columna_a_separar].quantile(0.25)
    q2 = df_a_separar[columna_a_separar].quantile(0.5)
    q3 = df_a_separar[columna_a_separar].quantile(0.75)
    qmax = df_a_separar[columna_a_separar].quantile(1)

    df_min_q1 = df_a_separar.query(
        f'`{columna_a_separar}` >= @qmin and `{columna_a_separar}` < @q1')
    df_q1_q2 = df_a_separar.query(
        f'`{columna_a_separar}` >= @q1 and `{columna_a_separar}` < @q2')
    df_q2_q3 = df_a_separar.query(
        f'`{columna_a_separar}` >= @q2 and `{columna_a_separar}` < @q3')
    df_q3_max = df_a_separar.query(
        f'`{columna_a_separar}` >= @q3 and `{columna_a_separar}` <= @qmax')

    return (df_min_q1, df_q1_q2, df_q2_q3, df_q3_max)

def rankear_y_plottear(df, eje_x, eje_y, titulo_grafico, numero_ranking):
    '''Esta función ordena los datos segun el eje y que se quiere plottear'''
    df_ordenada = df.sort_values(eje_y)
    diez_mas_altos = df.head(numero_ranking)
    diez_mas_bajos = df.tail(numero_ranking)

    mosaico = '''
    AB
    AC
    '''
    plot_args = [df, eje_x, eje_y, titulo_grafico]

    fig, axis = plt.subplot_mosaic(mosaico, figsize=(19.2, 20), layout='constrained')
    sns.barplot(data=plot_args[0], x=plot_args[1], y=plot_args[2], ax=axis['A'])

    axis['A'].xaxis.set_major_formatter('{x:,}')
    axis['A'].tick_params(axis='x', rotation=45)
    axis['A'].xaxis.tick_top()

    tabla_mas_altos = axis['B'].table(cellText=diez_mas_altos.values,
                                      colLabels=diez_mas_altos.columns,
                                      loc='center')
    tabla_mas_altos.scale(1, 2.3)
    tabla_mas_altos.auto_set_font_size(False)
    tabla_mas_altos.set_fontsize(10)


    tabla_mas_bajos = axis['C'].table(cellText=diez_mas_bajos.values,
                                      colLabels=diez_mas_bajos.columns,
                                      loc='center')
    
    tabla_mas_bajos.scale(1, 2.3)
    tabla_mas_bajos.auto_set_font_size(False)
    tabla_mas_bajos.set_fontsize(10)

    axis['B'].set_title('Top 10 gasto Neto')
    axis['C'].set_title('Bottom 10 gasto Neto')
    axis['B'].axis('off')
    axis['C'].axis('off')

    fig.suptitle(plot_args[3])
    plt.close()

    return fig

def analizar_distribucion_de_datos(df, columna_a_analizar, titulo_grafico):
    serie_columna = df[columna_a_analizar]
    descripcion_serie_columna = serie_columna.describe().to_frame().reset_index()
    descripcion_serie_columna = descripcion_serie_columna.rename(columns={'index': 'Estadisticas'})

    mosaico = '''
              AB
              AC
              '''

    fig, axis = plt.subplot_mosaic(mosaico, figsize=(19.2, 10.8), layout='constrained')
    args_plot = [df, columna_a_analizar, titulo_grafico]

    tabla = axis['A'].table(cellText=descripcion_serie_columna.values,
                            colLabels=descripcion_serie_columna.columns, loc='center')
    axis['A'].axis('off')

    sns.histplot(data=args_plot[0], x=args_plot[1], ax=axis['B'])
    sns.boxplot(data=args_plot[0], x=args_plot[1], ax=axis['C'])

    plt.ticklabel_format(style='plain', axis='x')

    axis['B'].tick_params(axis='x', rotation=45)
    axis['C'].tick_params(axis='x', rotation=45)
    axis['B'].xaxis.set_major_formatter('{x:,}')
    axis['C'].xaxis.set_major_formatter('{x:,}')

    fig.suptitle(args_plot[2])
    plt.close()

    return fig

def analisis_global_y_cuartil(df_agrupada, eje_x_agrupado):
    imagenes_a_guardar = {}
    dfs_a_guardar = {}

    fig_rank_global = rankear_y_plottear(
        df_agrupada, 'Porcentaje_salidas', eje_x_agrupado, 'Gasto por Servicio Global - Porcentaje_salidas', 10)
    fig_distribucion_global = analizar_distribucion_de_datos(
        df_agrupada, 'Porcentaje_salidas', 'Distribución Gasto Porcentaje_salidas por Servicio Global')
    imagenes_a_guardar['Global'] = [fig_rank_global, fig_distribucion_global]
    dfs_a_guardar['Global'] = [df_agrupada]

    cuartiles = separar_por_cuartil(df_agrupada, 'Porcentaje_salidas')
    for i, cuartil in enumerate(cuartiles):
        if not cuartil.empty:
            fig_rank_cuartil = rankear_y_plottear(
                cuartil, 'Porcentaje_salidas', eje_x_agrupado, f'Gasto por Servicio Intervalo Cuartil {i}', 10)
            fig_distribucion_cuartil = analizar_distribucion_de_datos(
                cuartil, 'Porcentaje_salidas', f'Distribución Gasto Neto Intervalo Cuartil {i}')
            imagenes_a_guardar[i] = [fig_rank_cuartil, fig_distribucion_cuartil]
            dfs_a_guardar[i] = [cuartil]
    
    return imagenes_a_guardar, dfs_a_guardar


def guardar_imagenes(imagenes_a_guardar, carpeta_a_guardar):
    for intervalo_imagen, lista_imagenes in imagenes_a_guardar.items():
        diccionario_intervalos = {'Global': 'Global', 0: 'INTERVALO_1_min_Q1',
                                  1: 'INTERVALO_2_Q1_Q2', 2: 'INTERVALO_3_Q2_Q3',
                                  3: 'INTERVALO_4_Q3_max'}
        path_nueva_carpeta = os.path.join(carpeta_a_guardar, diccionario_intervalos[intervalo_imagen])
        try:
            os.makedirs(path_nueva_carpeta)
        except FileExistsError:
            pass

        for i, imagen in enumerate(lista_imagenes):
            diccionario_nombres = {0: 'ranking', 1: 'distribucion'}
            tipo_archivo = diccionario_nombres[i]
            nombre_archivo = f'{diccionario_intervalos[intervalo_imagen]}_{tipo_archivo}.svg'
            ruta_archivo = os.path.join(path_nueva_carpeta, nombre_archivo)
            imagen.savefig(ruta_archivo)

def guardar_dfs(dfs_a_guardar, carpeta_a_guardar):
    for intervalo_df, lista_dfs in dfs_a_guardar.items():
        diccionario_intervalos = {'Global': 'Global', 0: 'INTERVALO_1_min_Q1',
                                  1: 'INTERVALO_2_Q1_Q2', 2: 'INTERVALO_3_Q2_Q3',
                                  3: 'INTERVALO_4_Q3_max'}
        
        path_nueva_carpeta = os.path.join(carpeta_a_guardar, diccionario_intervalos[intervalo_df])
        try:
            os.makedirs(path_nueva_carpeta)
        except FileExistsError:
            pass

        for df in lista_dfs:
            nombre_archivo = f'{diccionario_intervalos[intervalo_df]}.xlsx'
            ruta_archivo = os.path.join(path_nueva_carpeta, nombre_archivo)
            df.to_excel(ruta_archivo, index=False)



In [6]:
df = pd.concat(map(lambda x: pd.read_csv(x, parse_dates=[
               0], dayfirst=True), obtener_full_path('input')))
df_movimientos = df.sort_values('Fecha')
df_movimientos.columns = df_movimientos.columns.str.replace(' ', '_')


In [7]:
movimientos_entrada = df_movimientos.query('Movimiento == "Entrada"')
movimientos_salida = df_movimientos.query('Movimiento == "Salida"')

suma_entradas = movimientos_entrada.groupby(by=['Codigo_Articulo', 'Nombre']).sum().reset_index()
suma_salidas = movimientos_salida.groupby(by=['Codigo_Articulo', 'Nombre']).sum().reset_index()

suma_entradas['Tipo_Movimiento'] = 'Entrada'
suma_salidas['Tipo_Movimiento'] = 'Salida'

In [8]:
entradas_salidas = pd.merge(suma_entradas, suma_salidas, how='outer', on='Codigo_Articulo',
                            suffixes=('_entradas', '_salidas'))

entradas_salidas = entradas_salidas[[
    'Codigo_Articulo', 'Nombre_entradas', 'Nombre_salidas', 'Cantidad_entradas', 'Cantidad_salidas']]

entradas_salidas['Porcentaje_salidas'] = entradas_salidas['Cantidad_salidas'] / entradas_salidas['Cantidad_entradas']
entradas_salidas = entradas_salidas.sort_values('Porcentaje_salidas')
entradas_salidas_validas = entradas_salidas.dropna(subset='Porcentaje_salidas') \
                                           .drop(columns=['Nombre_salidas'])

entradas_salidas_validas = entradas_salidas_validas.round(2)
entradas_salidas_validas['Nombre_entradas'] = entradas_salidas_validas['Nombre_entradas'].str[:40]


In [9]:
bajo_100 = entradas_salidas_validas.query('Porcentaje_salidas < 1')
sobre_100 = entradas_salidas_validas.query('Porcentaje_salidas >= 1')

In [10]:
imagenes, dfs = analisis_global_y_cuartil(bajo_100, eje_x_agrupado='Nombre_entradas')
carpeta_a_guardar = os.path.join('output', 'ranking_salida_articulos_bajo_100')
guardar_imagenes(imagenes, carpeta_a_guardar=carpeta_a_guardar)
guardar_dfs(dfs, carpeta_a_guardar=carpeta_a_guardar)
    

In [11]:
imagenes_sobre_100, dfs_sobre_100 = analisis_global_y_cuartil(sobre_100, eje_x_agrupado='Nombre_entradas')
carpeta_a_guardar = os.path.join('output', 'ranking_salida_articulos_sobre_100')
guardar_imagenes(imagenes_sobre_100, carpeta_a_guardar=carpeta_a_guardar)
guardar_dfs(dfs_sobre_100, carpeta_a_guardar=carpeta_a_guardar)

C:\Users\gmolina\AppData\Local\Temp\ipykernel_33372\3882483041.py:139: UserWarning: Glyph 9 (	) missing from current font.
  imagen.savefig(ruta_archivo)
